In [2]:
# Core
import numpy as np
import rebound

In [8]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_utils import make_sim_planets, make_archive

In [25]:
# Typing
from typing import Tuple

In [19]:
# Make sim with planets
epoch = 59000
sim_epoch = make_sim_planets(epoch=epoch)

In [9]:
# Date range to process
mjd0: int = epoch-100
mjd1: int = epoch+100

In [10]:
# Shared time_step and save_step
time_step: int = 1
save_step: int = 1

# Flags for building simulation archive
save_elements: bool = False
progbar: bool = True

In [15]:
# # Make simulation archive
# sa_planets = make_archive(sim_epoch=sim, 
#                  mjd0=mjd0, mjd1=mjd1, time_step=time_step, save_step=save_step,
#                  save_elements=save_elements, progbar=progbar)

In [20]:
# Look up the epoch from the base simulation
epoch: int = sim_epoch.epoch

# Look up body IDs and names
body_ids: np.ndarray = sim_epoch.body_ids
body_names: np.ndarray = sim_epoch.body_names

In [21]:
body_ids

array([ 10,   1,   2, 399, 301,   4,   5,   6,   7,   8,   9])

In [22]:
body_names

array(['Sun', 'Mercury Barycenter', 'Venus Barycenter', 'Earth', 'Moon',
       'Mars Barycenter', 'Jupiter Barycenter', 'Saturn Barycenter',
       'Uranus Barycenter', 'Neptune Barycenter', 'Pluto Barycenter'],
      dtype=object)

In [23]:
# This way, time is indexed from t0=0 to t1 = (dt1-dt0)
t0: float = 0.0
t1: float = mjd1 - mjd0
t_epoch: float = float(epoch) - mjd0

# Create copies of the simulation to integrate forward and backward
sim_fwd: rebound.Simulation = sim_epoch.copy()
sim_back: rebound.Simulation = sim_epoch.copy()

# Set the time counter on both simulation copies to the epoch time
sim_fwd.t = t_epoch
sim_back.t = t_epoch
# Flip sign of dt on sim_back
sim_back.dt *= -1.0

# Set the times for snapshots in both directions;
ts: np.array = np.arange(t0, t1+time_step, time_step)
idx: int = np.searchsorted(ts, t_epoch, side='left')
ts_fwd: np.array = ts[idx:]
ts_back: np.array = ts[:idx][::-1]

# The epochs corresponding to the times in ts; as MJD and as datetime
epochs: np.array = ts + (epoch - t_epoch)

In [26]:
# Number of snapshots
M_back: int = len(ts_back)
M_fwd: int = len(ts_fwd)
M: int = M_back + M_fwd
# Number of particles
N: int = sim_epoch.N

# Initialize arrays for the position and velocity
shape_qv: Tuple[int] = (M, N, 3)
q: np.array = np.zeros(shape_qv, dtype=np.float64)
v: np.array = np.zeros(shape_qv, dtype=np.float64)